IMPORTING THE LIBRARIRES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import warnings
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,StratifiedKFold

IMPORTING THE DATASETS

In [ ]:
dataset=pd.read_csv('project 5 dataset.csv')
train_df=pd.read_csv('project 5(train).csv')
test_df=pd.read_csv('project 5(test).csv')
dataset

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
dataset

In [ ]:
train_df.info()

CATEGORIZATION OF SETS

In [ ]:
columns = train_df.columns.value_counts()

dtype_counts = train_df.dtypes.value_counts()
myexplode=(0,0,0.1)

categorical_Features = len(train_df.columns[[1,2,4,11]])
Numerical_Features = len(train_df.columns[[5,9,6,7]])
Mixed_dtype_Features= len(train_df.columns[[8,10]])

In [ ]:
counts = [categorical_Features,Numerical_Features,Mixed_dtype_Features]
labels = ['Categorical','Numerical','Mixed_dtype']

PIE CHART REPRESENTATION OF CATEGORIES

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(16,4))
axes[0].pie(columns, labels=columns.index,autopct='%1.1f%%',startangle=0)
axes[0].set_title('Features of a Titanic dataset')
axes[0].set_aspect('equal')

axes[1].pie(dtype_counts,labels=dtype_counts.index,explode=myexplode,autopct='%1.1f%%',startangle=50)
axes[1].set_title('dtypes of Features in a Titanic dataset')
axes[1].set_aspect('equal')

axes[2].pie(counts,labels=labels,autopct='%1.1f%%',explode=myexplode,startangle=50)
axes[2].set_title('Data distribution of Features')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

In [ ]:
train_df.describe()

HISTOGRAM REPRESTATION

In [ ]:
train_df.iloc[:,1:].hist(figsize= (6,6))

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df['Age'].describe()

In [ ]:
train_df.head()

In [ ]:
train_df['Name'].head()

HONOURIFIC WORDS SPLITTING FROM SETS

In [ ]:
title = [i.split(',')[1].split('.')[0].strip() for i in train_df['Name']]
train_df['title'] = pd.Series(title)
train_df['title'].head()

PLOTS FOR THE TITLES

In [ ]:
value_counts = train_df['title'].value_counts()

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Subplot 1: Text Display of Value Counts as a Table
cell_text = [[title, count] for title, count in value_counts.items()]
ax1.table(cellText=cell_text, colLabels=['Title', 'Count'], cellLoc='center', loc='center')
ax1.axis('off')  # Turn off axes for text output
ax1.set_title('Value Counts')

# Subplot 2: Count Plot as Bar Plot
sns.countplot(data=train_df, x='title', ax=ax2)
ax2.set_title('Count Plot')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=40, ha='right')

plt.show()

In [ ]:
train_df["title"] = train_df["title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train_df['title'] = train_df['title'].replace(['Miss','Ms','Mme','Mlle','Mrs'],'Miss/Ms/Mme/Mlle/Mrs')
sns.set_theme(style="darkgrid")
custom_palette = ["#F08080", "#90EE90", "#ADD8E6", "#F8B1D0"]
train_df['title'].value_counts().plot(kind='bar',color=custom_palette,width=0.3)
plt.title('Distribution of title Feature')
plt.xticks(rotation=0)
plt.xlabel('')
plt.ylabel('count')
plt.show()

In [ ]:
train_df.loc[(train_df['title'] == 'Mr') & (train_df['Age'].isnull()), 'Age'] = train_df[train_df['title'] == 'Mr']['Age'].median()
train_df.loc[(train_df['title'] == 'Miss/Ms/Mme/Mlle/Mrs') & (train_df['Age'].isnull()), 'Age'] = train_df[train_df['title'] == 'Miss/Ms/Mme/Mlle/Mrs']['Age'].median()
train_df.loc[(train_df['title'] == 'Master') & (train_df['Age'].isnull()), 'Age'] = train_df[train_df['title'] == 'Master']['Age'].median()
train_df.loc[(train_df['title'] == 'Rare') & (train_df['Age'].isnull()), 'Age'] = train_df[train_df['title'] == 'Rare']['Age'].median()


In [ ]:
train_df.isnull().sum()

In [ ]:
train_df['Pclass'].value_counts()

DESCRIPTION OF FARE RESPECT YO THE PASSENGER CLASS

In [ ]:
print('Fare with respect to Passenger class:')
print(train_df.loc[train_df['Pclass'] == 1]['Fare'].describe())
print('='*25)
print(train_df.loc[train_df['Pclass'] == 2]['Fare'].describe())
print('='*25)
print(train_df.loc[train_df['Pclass'] == 3]['Fare'].describe())

PASSENGER CLASS 1 SURVIVALS AND NON SURVIVALS

In [ ]:
p1_survival = train_df[(train_df['Pclass'] == 1) & (train_df['Fare'] <= 512.329200)]
print(p1_survival['Survived'].value_counts())
print(p1_survival[p1_survival['Survived'] == 0]['Cabin'].value_counts())
print(p1_survival[p1_survival['Survived'] == 1]['Cabin'].value_counts())

DIED AND naN MEMBERS IN CLASS 1

In [ ]:
p1_died_null_cabin = p1_survival[(p1_survival['Survived'] == 0) & (p1_survival['Cabin'].isnull())]
for i in p1_died_null_cabin.index.to_list():
    if i<=5:
        train_df.loc[i,'Cabin'] = 'C30'
    elif (i>5) & (i<=10):
        train_df.loc[i,'Cabin'] = 'C110'
    elif (i>10) & (i<=15):
        train_df.loc[i,'Cabin'] = 'B94'
    else:
        train_df.loc[i,'Cabin'] = 'E44'

SURVIVED AND naN MEMBERS IN CLASS 1

In [ ]:
p1_alive_null_cabin = p1_survival[(p1_survival['Survived'] ==1) & (p1_survival['Cabin'].isnull())]
for i in p1_alive_null_cabin.index.to_list():
    if i>=5:
        train_df.loc[i,'Cabin'] = 'B96 B98'
    elif (i>5) & (i<=10):
        train_df.loc[i,'Cabin'] = 'E40'
    elif (i>10) & (i<=15):
        train_df.loc[i,'Cabin'] = 'E34'
    else:
        train_df.loc[i,'Cabin'] = 'C7'


TESTING OF UNIQUE MEMBER

In [ ]:
train_df.iloc[187]


PASSENGER CLASS 2 SURVIVALS AND NON SURVIVALS

In [ ]:
p2_survival = train_df[(train_df['Pclass'] == 2) & (train_df['Fare'] <= 73.500000)]
print(p2_survival['Survived'].value_counts())
print(p2_survival[p2_survival['Survived'] == 0]['Cabin'].value_counts())
print(p2_survival[p2_survival['Survived'] == 1]['Cabin'].value_counts())

DIED AND naN MEMBERS IN CLASS 1

In [ ]:
p2_died_null_cabin =p2_survival[(p2_survival['Survived'] == 0) & (p2_survival['Cabin'].isnull())]
for i in p2_died_null_cabin.index.to_list():
    if i<=61:
        train_df.loc[i,'Cabin'] = 'F2'
    elif (i>61) & (i<=122):
        train_df.loc[i,'Cabin'] = 'D'
    else:
        train_df.loc[i,'Cabin'] = 'E77'

SURVIVED AND naN MEMBERS IN CLASS 2

In [ ]:
p2_alive_null_cabin = p2_survival[(p2_survival['Survived'] == 1) & (p2_survival['Cabin'].isnull())]
for i in p2_alive_null_cabin.index.to_list():
    if i<=15:
        train_df.loc[i,'Cabin'] = 'F33'
    elif (i>15) & (i<=30):
        train_df.loc[i,'Cabin'] = 'E101'
    elif (i>30) & (i<=40):
        train_df.loc[i,'Cabin'] = 'F4'
    elif (i>40) & (i<=50):
        train_df.loc[i,'Cabin'] = 'F2'
    elif (i>50) & (i<=60):
        train_df.loc[i,'Cabin'] = 'D'
    else:
        train_df.loc[i,'Cabin'] = 'D56'

PASSENGER CLASS 3 SURVIVALS AND NON SURVIVALS

In [ ]:
p3_survival = train_df[(train_df['Pclass'] == 3) & (train_df['Fare'] <= 69.550000)]
print(p3_survival['Survived'].value_counts())
print(p3_survival[p3_survival['Survived'] == 0]['Cabin'].value_counts())
print(p3_survival[p3_survival['Survived'] == 1]['Cabin'].value_counts())

DIED AND naN MEMBERS IN CLASS 3

In [ ]:
p3_died_null_cabin = p3_survival[(p3_survival['Survived'] == 0) & (p3_survival['Cabin'].isnull())]
for i in p3_died_null_cabin.index.to_list():
    if i<=122:
        train_df.loc[i,"Cabin"]="G6"
    elif (i>122) & (i<=244):
        train_df.loc[i,'Cabin']='G6'
    elif (i>244) & (i<=305):
        train_df.loc[i,'Cabin']='F G63'
    else:
        train_df.loc[i,'Cabin']='F38'

SURVIVED AND naN MEMBERS IN CLASS 3

In [ ]:
p3_alive_null_cabin = p3_survival[(p3_survival['Survived'] == 1) & (p3_survival['Cabin'].isnull())]
for i in p3_alive_null_cabin.index.to_list():
    if i<=38:
        train_df.loc[i,"Cabin"]="F G73"
    elif (i>38) & (i<=76):
        train_df.loc[i,'Cabin']='E121'
    elif (i>76) & (i<=95):
        train_df.loc[i,'Cabin']='F E69'
    else:
        train_df.loc[i,'Cabin']='E10'

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df['Embarked'].value_counts()

REPLACING naN ROWS IN EMBARKED COLUMN IN A DATASET WITH Q

In [ ]:
train_df['Embarked'].replace(np.nan,'Q',inplace=True)
train_df.isnull().sum()

In [ ]:
train_df.duplicated().sum()

In [ ]:
train_df['Survived'].value_counts()

ENCODING OF THE AGE AND FARE SECTION

In [ ]:

from sklearn.preprocessing import RobustScaler,LabelEncoder
col_headers=['Age','Fare']
scl = RobustScaler()
train_df[col_headers] = scl.fit_transform(train_df[col_headers])
train_df.head()

ENCODING OF THE TEXT DESCRIPTIVE COLUMNS IN THE DATASET

In [ ]:
column_headers = ['Name','Sex', 'Ticket', 'Cabin', 'Embarked','title']
le = LabelEncoder()
train_df[column_headers] = train_df[column_headers].apply(le.fit_transform)
train_df.head()

CORRELATION MATRIX

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(train_df.corr(),annot=True,fmt='.2f')

In [ ]:
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']

LOGICAL REGRESSION MODELLING

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm=print(cm)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
classification_rep = classification_report(y_test, y_pred)
classification_rep

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
X, y = make_blobs(n_samples=100, centers=2, n_features=100, cluster_std=20)
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

K-NN MODELLING

In [ ]:
model = KNeighborsClassifier(n_neighbors = 100, metric = 'minkowski', p = 1)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm=print(cm)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
X, y = make_blobs(n_samples=1000, centers=2, n_features=100, cluster_std=20)
model = KNeighborsClassifier()
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

SVM MODELLING

In [ ]:
model = SVC(kernel = 'linear', random_state = 0)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm=print(cm)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
model = SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']
grid = dict(kernel=kernel,C=C,gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

KERNAL SVM MODELLING

In [ ]:
model = SVC(kernel = 'rbf', random_state = 0)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm=print(cm)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
model = SVC()
kernel = ['poly', 'rbf', 'sigmoid','linear']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']
grid = dict(kernel=kernel,C=C,gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

DECISION TREE MODELLING

In [ ]:
model = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm=print(cm)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm=print(cm)

In [ ]:
accuracy_score(y_test,y_pred)

RANDOM FOREST MODELLING

In [ ]:
model = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 0)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm=print(cm)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
X, y = make_blobs(n_samples=100, centers=2, n_features=100, cluster_std=20)
model = RandomForestClassifier()
n_estimators = [10, 100, 1000]
max_features = ['sqrt', 'log2']
grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
     print("%f (%f) with: %r" % (mean, stdev, param))

GRADIENT BOOSTING MODELLING

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=300,
                                 learning_rate=0.05,
                                 random_state=100,
                                 max_features=5 )
gbc.fit(X_train,y_train)
y_pred=gbc.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm=confusion_matrix(y_test,y_pred)
cm=print(cm)

In [ ]:
accuracy_score(y_test,y_pred)

XG BOOST MODELLING

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

FEATURE SCALING

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train

In [ ]:
X_test